In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

### 1. Load Data

In [2]:
homepage = pd.read_csv('/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/data/Funnel/home_page_table.csv')
confirm = pd.read_csv('/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/data/Funnel/payment_confirmation_table.csv')
payment = pd.read_csv('/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/data/Funnel/payment_page_table.csv')
search = pd.read_csv('/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/data/Funnel/search_page_table.csv')
user = pd.read_csv('/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/data/Funnel/user_table.csv')

In [3]:
#check tables' format
file_list = [user, homepage, confirm, payment, search]
for i in file_list:
    print(i.head(3))

   user_id        date   device     sex
0   450007  2015-02-28  Desktop  Female
1   756838  2015-01-13  Desktop    Male
2   568983  2015-04-09  Desktop    Male
   user_id       page
0   313593  home_page
1   468315  home_page
2   264005  home_page
   user_id                       page
0   123100  payment_confirmation_page
1   704999  payment_confirmation_page
2   407188  payment_confirmation_page
   user_id          page
0   253019  payment_page
1   310478  payment_page
2   304081  payment_page
   user_id         page
0    15866  search_page
1   347058  search_page
2   577020  search_page


In [4]:
file_list = [user, homepage, search, payment, confirm]
for i in file_list:
    print(i.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90400 entries, 0 to 90399
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  90400 non-null  int64 
 1   date     90400 non-null  object
 2   device   90400 non-null  object
 3   sex      90400 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90400 entries, 0 to 90399
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  90400 non-null  int64 
 1   page     90400 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45200 entries, 0 to 45199
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  45200 non-null  int64 
 1   page     45200 non-null  object
dtypes: int64(1), object(1)
memory usage: 706.4+

### 2. Merge data and calculate basic conversion rate

In [5]:
# change all 'page' column value into 1, which indicates that users convert into next level of funnel 
file_list = [homepage, search, payment, confirm]
for i in file_list:
    i['page'] = 1

In [6]:
file_list = [homepage, search, payment, confirm]
for i in file_list:
    user = pd.merge(left = user, right = i, how = 'left', 
                     left_on = 'user_id', right_on = 'user_id')
user.columns = ['user_id', 'date', 'device', 'sex', 
                 'homepage', 'search', 'payment', 'confirm']
user = user.fillna(0)

In [7]:
user.head(3)

,user_id,date,device,sex,homepage,search,payment,confirm
0,450007,2015-02-28,Desktop,Female,1,0.0,0.0,0.0
1,756838,2015-01-13,Desktop,Male,1,0.0,0.0,0.0
2,568983,2015-04-09,Desktop,Male,1,1.0,0.0,0.0


In [8]:
# Conversion rate from homepage to next page
# for example: 50% of all users reach out the search_page, 
# but only 0.5% of all users reach out the payment confirmation page

for i in ['homepage', 'search', 'payment', 'confirm']:
    print('homepage -->', i, round(np.mean(user[i]), 4))

homepage --> homepage 1.0
homepage --> search 0.5
homepage --> payment 0.0667
homepage --> confirm 0.005


In [9]:
# Conversion rate from current page to next page
# for example: 50% of all users in homepage reach out the search page, 
# but only 13% of all users in search_page reach out the payment page

print('homepage --> search_page',  round(user[(user['homepage'] == 1)]['search'].mean(), 4))
print('search_page --> payment_page',  round(user[(user['search'] == 1)]['payment'].mean(), 4))
print('payment_page --> confirm', round(user[(user['payment'] == 1)]['confirm'].mean(), 4))

homepage --> search_page 0.5
search_page --> payment_page 0.1334
payment_page --> confirm 0.075


In [10]:
# Conversion rate from current page to confirm page
# for example: 0.5% of all users in homepage reach out the payment_confrimation page, 
# but 1% of all users in search_page reach out the payment_confirmation page

for i in ['homepage', 'search', 'payment']:
    print(i, '--> confirm', round(user[(user[i] == 1)]['confirm'].mean(), 4))

homepage --> confirm 0.005
search --> confirm 0.01
payment --> confirm 0.075


The general conversion rate displayed above indicates that the users from payment page to confirm page is extremely low, which is abnormal.

### 3. Conversion rate by device

In [12]:
temp_1 = user[(user['homepage'] == 1)].groupby('device')['search'].mean().reset_index()
temp_2 = user[(user['search'] == 1)].groupby('device')['payment'].mean().reset_index()
temp_3 = user[(user['payment'] == 1)].groupby('device')['confirm'].mean().reset_index()
device_summary = temp_1.merge(temp_2, on = 'device').merge(temp_3, on = 'device').T
device_summary.columns = device_summary.iloc[0]
device_summary = device_summary.iloc[1:]
device_summary

device,Desktop,Mobile
search,0.5,0.5
payment,0.1,0.2
confirm,0.049834,0.1


The performance of mobile is better than the desktop, which means that the company needs to improve the desktop design. 

Meanwhile, it's strange that the conversion rate from payment page to confirmation page is very low. That is to say many customers change their mind to confirm the final order click.

In [13]:
temp_1 = user[(user['homepage'] == 1)].groupby('sex')['search'].mean().reset_index()
temp_2 = user[(user['search'] == 1)].groupby('sex')['payment'].mean().reset_index()
temp_3 = user[(user['payment'] == 1)].groupby('sex')['confirm'].mean().reset_index()
sex_summary = temp_1.merge(temp_2, on = 'sex').merge(temp_3, on = 'sex').T
sex_summary.columns = sex_summary.iloc[0]
sex_summary = sex_summary.iloc[1:]
sex_summary

sex,Female,Male
search,0.503073,0.496944
payment,0.136708,0.130083
confirm,0.077742,0.072014


There is no significant difference between female and male users in conversion rate